In [2]:
import pandas as pd
import duckdb
from cryptocmd import CmcScraper

In [3]:
j = 0
df2 = pd.DataFrame()

df1 = pd.read_csv('/Users/denistkachenko/Documents/nft_sales.csv')

/var/folders/1w/5fknmcjj7_g7sfbf2lffwy1h0000gn/T/ipykernel_39249/4191991506.py:4: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('/Users/denistkachenko/Documents/nft_sales.csv')


In [4]:
values = duckdb.query("""

SELECT
 distinct event_payment_symbol as p_method
 From df1


                    """).df()
crypto_list = values["p_method"].values.tolist()
for i in crypto_list:
    try:
        scraper = CmcScraper(str(crypto_list[j]),"01-02-2021", "28-02-2021")

        df3 = scraper.get_dataframe()
        df3["token"]=crypto_list[j]
        df2 = pd.concat([df3, df2], ignore_index=True)
        j += 1
    except:
        pass

In [13]:
main = duckdb.query("""

with max as (

SELECT
  asset_id

 --,event_payment_symbol as p_method_high
 ,min(date_trunc('day',cast(event_time as timestamp ))) as high_date
 ,max((df2.Low + df2.High)/2*event_total_price) as max_price
 --,count(asset_id)
 --,count(event_id) as trade_amount
 From df1
 JOIN df2 on df2.Date = date_trunc('day', cast(df1.event_time as TIMESTAMP )) and df2.token = df1.event_payment_symbol
 --where asset_id ='18218837'
 group by 1

),
max_date as (

select distinct df1.asset_id
 ,date_trunc('day',cast(df1.event_time as timestamp )) as high
 ,max.max_price
from df1
JOIN df2 on df2.Date = date_trunc('day', cast(df1.event_time as TIMESTAMP )) and df2.token = df1.event_payment_symbol
inner join max on max.max_price = (df2.Low + df2.High)/2*df1.event_total_price and max.asset_id = df1.asset_id


),
min as (
SELECT
  asset_id
 ,min((df2.Low + df2.High)/2*event_total_price) as min_price
 From df1
 JOIN df2 on df2.Date = date_trunc('day', cast(df1.event_time as TIMESTAMP )) and df2.token = df1.event_payment_symbol
 --where asset_id ='18218837'
 group by 1),

 min_date as (

select distinct df1.asset_id
 ,date_trunc('day',cast(df1.event_time as timestamp )) as low
 ,min.min_price
from df1
JOIN df2 on df2.Date = date_trunc('day', cast(df1.event_time as TIMESTAMP )) and df2.token = df1.event_payment_symbol
inner join min on min.min_price = (df2.Low + df2.High)/2*df1.event_total_price and min.asset_id = df1.asset_id),


main as (
select *
from max_date max
join min_date min on min.asset_id = max.asset_id
--where max.high_date < min.low_date
WHERE max_price != min_price
),
best as (
select
 low
 ,count(asset_id)
 from main
 group by 1
    ),

worst as (
select
 high
 ,count(asset_id)
 from main
 group by 1
), data as (
select *,
case when high < low then min_price-max_price else max_price-min_price end as delta
from main)
select case when delta <0 then 'lost value' else 'gained value' end as status
,count(delta)
from data
group by 1

                    """).df()

In [14]:
main

,status,count(delta)
0,gained value,10530
1,lost value,2266


In [15]:
data2 = duckdb.query("""
select collection_name
,count(event_id) as trades
,avg((df2.Low + df2.High)/2*event_total_price) as price
from df1
JOIN df2 on df2.Date = date_trunc('day', cast(df1.event_time as TIMESTAMP )) and df2.token = df1.event_payment_symbol
group by 1
order by trades desc, price desc


""").df()

In [16]:
data2

,collection_name,trades,price
0,Sorare,40679,238.525753
1,Rarible,11695,795.674802
2,Hashmasks,7580,4218.005046
3,Axie Infinity,5943,273.447740
4,CryptoKitties,4765,169.588297
...,...,...,...
1528,Aarav M,1,0.000000
1529,Hannih,1,0.000000
1530,$2021STUFF$,1,0.000000
1531,Fortuna,1,0.000000


In [17]:
data3 = duckdb.query("""
select collection_name
,asset_contract_date

,count(event_id) as trades
,avg((df2.Low + df2.High)/2*event_total_price) as price
from df1
JOIN df2 on df2.Date = date_trunc('day', cast(df1.event_time as TIMESTAMP )) and df2.token = df1.event_payment_symbol
group by 1,2
order by trades desc, price desc
""").df()

In [18]:
data3


,collection_name,asset_contract_date,trades,price
0,Sorare,2019-07-04T10:01:29.425402,40679,238.525753
1,Rarible,2020-05-27T16:53:32.834583,10245,733.586178
2,Hashmasks,2021-01-28T10:22:01.756793,7580,4218.005046
3,Axie Infinity,2018-04-16T05:57:44.916272,5943,273.447740
4,CryptoKitties,2018-01-23T04:51:38.832339,4765,169.588297
...,...,...,...,...
1646,Crypto Tittiez,2019-11-07T10:39:59.171966,1,0.000000
1647,$2021STUFF$,2020-12-02T17:40:53.232025,1,0.000000
1648,Cryptomaniacfr | HOlO10,2020-12-02T17:40:53.232025,1,0.000000
1649,Little Daniela,2020-12-02T17:40:53.232025,1,0.000000
